# **Project: Healthcare answer summarization**

# 1 . Dataset Analysis :

## 1.1 Training Dataset :

In [10]:
import json
from collections import Counter, defaultdict

# Load the dataset
with open('/kaggle/input/palsma/train.json', 'r') as f:
    train_data = json.load(f)

# Initialize counters
label_counter = Counter()
summary_type_counter = Counter()
questions_with_labels = defaultdict(set)

# Analyze the dataset
for item in train_data:
    uri = item.get("uri")
    question = item.get("question")
    
    # Count labelled answer spans
    labelled_spans = item.get("labelled_answer_spans", {})
    for label_type, spans in labelled_spans.items():
        label_counter[label_type] += len(spans)
        questions_with_labels[label_type].add(uri)
    
    # Count summary types
    summaries = item.get("labelled_summaries", {})
    for summary_type in summaries:
        summary_type_counter[summary_type] += 1

# Report
print("Total number of questions:", len(train_data))
print("\nLabelled Answer Span Counts:")
for label, count in label_counter.items():
    print(f"{label}: {count} (in {len(questions_with_labels[label])} questions)")

print("\nSummary Types Counts:")
for summary, count in summary_type_counter.items():
    print(f"{summary}: {count}")


Total number of questions: 2236

Labelled Answer Span Counts:
INFORMATION: 4388 (in 1767 questions)
SUGGESTION: 3613 (in 1360 questions)
CAUSE: 579 (in 308 questions)
EXPERIENCE: 1245 (in 747 questions)
QUESTION: 284 (in 215 questions)

Summary Types Counts:
INFORMATION_SUMMARY: 1742
CAUSE_SUMMARY: 305
SUGGESTION_SUMMARY: 1363
EXPERIENCE_SUMMARY: 745
QUESTION_SUMMARY: 213


## 1.2 Validation Dataset:

In [8]:
import json
from collections import Counter, defaultdict

# Load the dataset
with open('/kaggle/input/palsma/valid.json', 'r') as f:
    valid_data = json.load(f)

# Initialize counters
label_counter = Counter()
summary_type_counter = Counter()
questions_with_labels = defaultdict(set)

# Analyze the dataset
for item in valid_data:
    uri = item.get("uri")
    question = item.get("question")
    
    # Count labelled answer spans
    labelled_spans = item.get("labelled_answer_spans", {})
    for label_type, spans in labelled_spans.items():
        label_counter[label_type] += len(spans)
        questions_with_labels[label_type].add(uri)
    
    # Count summary types
    summaries = item.get("labelled_summaries", {})
    for summary_type in summaries:
        summary_type_counter[summary_type] += 1

# Report
print("Total number of questions:", len(valid_data))
print("\nLabelled Answer Span Counts:")
for label, count in label_counter.items():
    print(f"{label}: {count} (in {len(questions_with_labels[label])} questions)")

print("\nSummary Types Counts:")
for summary, count in summary_type_counter.items():
    print(f"{summary}: {count}")


Total number of questions: 959

Labelled Answer Span Counts:
EXPERIENCE: 565 (in 316 questions)
INFORMATION: 1805 (in 735 questions)
SUGGESTION: 1635 (in 595 questions)
CAUSE: 266 (in 139 questions)
QUESTION: 131 (in 102 questions)

Summary Types Counts:
EXPERIENCE_SUMMARY: 315
INFORMATION_SUMMARY: 733
CAUSE_SUMMARY: 138
SUGGESTION_SUMMARY: 595
QUESTION_SUMMARY: 101


## 1.3 Test Dataset :

In [9]:
import json
from collections import Counter, defaultdict

# Load the dataset
with open('/kaggle/input/palsma/test.json', 'r') as f:
    test_data = json.load(f)

# Initialize counters
label_counter = Counter()
summary_type_counter = Counter()
questions_with_labels = defaultdict(set)

# Analyze the dataset
for item in test_data:
    uri = item.get("uri")
    question = item.get("question")
    
    # Count labelled answer spans
    labelled_spans = item.get("labelled_answer_spans", {})
    for label_type, spans in labelled_spans.items():
        label_counter[label_type] += len(spans)
        questions_with_labels[label_type].add(uri)
    
    # Count summary types
    summaries = item.get("labelled_summaries", {})
    for summary_type in summaries:
        summary_type_counter[summary_type] += 1

# Report
print("Total number of questions:", len(test_data))
print("\nLabelled Answer Span Counts:")
for label, count in label_counter.items():
    print(f"{label}: {count} (in {len(questions_with_labels[label])} questions)")

print("\nSummary Types Counts:")
for summary, count in summary_type_counter.items():
    print(f"{summary}: {count}")


Total number of questions: 640

Labelled Answer Span Counts:
INFORMATION: 1188 (in 488 questions)
CAUSE: 197 (in 103 questions)
SUGGESTION: 1105 (in 394 questions)
EXPERIENCE: 374 (in 207 questions)
QUESTION: 86 (in 65 questions)

Summary Types Counts:
INFORMATION_SUMMARY: 486
CAUSE_SUMMARY: 102
SUGGESTION_SUMMARY: 394
EXPERIENCE_SUMMARY: 206
QUESTION_SUMMARY: 64


In [11]:
def print_samples(data, name="Dataset", count=5):
    print(f"\n{'='*20} {name} Samples {'='*20}")
    for i, sample in enumerate(data[:count]):
        print(f"\n--- Sample {i+1} ---")
        print("Question:", sample.get("question"))
        print("Answers:", sample.get("answers", []))
        summaries = sample.get("labelled_summaries", {})
        if summaries:
            print("Summaries:")
            for k, v in summaries.items():
                print(f"  {k}: {v}")
        else:
            print("Summaries: None")



# Print samples
print_samples(train_data, name="Train")
print_samples(valid_data, name="Validation")
print_samples(test_data, name="Test")



==================== Train Samples ====================

--- Sample 1 ---
Question: what is parkinesonism?
Answers: ['u spelt it wrong !!\nParkinson\'s disease is one of the most common neurologic disorders of the elderly. The term "parkinsonism" refers to any condition that causes any combination of the types of movement abnormalities seen in Parkinson\'s disease by damaging or destroying dopamine neurons in a certain area of the brain.', "Parkinsonism describes the common symptoms of Parkinson's disease - tremor, rigidity, akinesia or bradykinesia and postural instability. Those patients who respond to drug treatment for Parkinson's disease are diagnosed with it, and those who do not have parkinsonism."]
Summaries:
  INFORMATION_SUMMARY: Parkinson's disease is a prevalent neurologic disorder among the elderly. The term "parkinsonism" encompasses any condition leading to movement abnormalities similar to those observed in Parkinson's disease. This condition arises from the damage or 

# 2 . Training Models :

In [2]:
pip install rouge

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

# 2.1 Bart Model : 

In [7]:
import os
import json
import math
import torch
import warnings
import numpy as np
from tqdm import tqdm
from rouge import Rouge
from scipy.spatial.distance import cosine
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import (
    BertTokenizer, BertModel, RobertaTokenizer, RobertaForSequenceClassification,
    AutoTokenizer, AutoModelForSeq2SeqLM
)
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW


warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### =============================== Dataset ===============================
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item.get("question", "").strip()
        answers = item.get("answers", [])

        labelled_summary_dict = item.get("labelled_summaries", {})
        
        if not labelled_summary_dict:
            return self.__getitem__((idx + 1) % len(self.data))  # Skip bad sample


        labelled_answer_spans = item.get("labelled_answer_spans", {})

        
        # Assume only one perspective (e.g., "INFORMATION")
        # if not labelled_summary_dict:
        #     raise ValueError("Missing labelled_summaries in example.")
        perspective_key = list(labelled_summary_dict.keys())[0]
        perspective = perspective_key.replace("_SUMMARY", "")
        target_text = labelled_summary_dict[perspective_key].strip()

        # Prepare answer context
        concatenated_answers = " ".join([ans.replace('\n', ' ').strip() for ans in answers])

        # Definitions and tones for guidance
        start_phrases = {
            "SUGGESTION": ("It is suggested", "Advisory, Recommending",
                           ["Advisory", "Recommending", "Cautioning", "Prescriptive", "Guiding"]),
            "INFORMATION": ("For information purposes", "Informative, Educational",
                            ["Clinical", "Scientific", "Informative", "Educational"]),
            "EXPERIENCE": ("In user's experience", "Personal, Narrative",
                           ["Personal", "Narrative", "Introspective", "Exemplary"]),
            "CAUSE": ("Some of the causes", "Explanatory, Causal",
                      ["Diagnostic", "Explanatory", "Causal", "Due to"]),
            "QUESTION": ("It is inquired", "Seeking Understanding",
                         ["Inquiry", "Rhetorical", "Exploratory Questioning"])
        }

        definitions = {
            "SUGGESTION": "Advice or recommendations to assist users.",
            "INFORMATION": "Knowledge about diseases and facts.",
            "EXPERIENCE": "Individual experiences or insights.",
            "CAUSE": "Reasons responsible for symptoms or conditions.",
            "QUESTION" : "Inquiry made for deeper understanding."
        }

        start_with, tone, _ = start_phrases.get(perspective, ("", "", []))
        definition = definitions.get(perspective, "")

        # Check and prepend start phrase if necessary
        if len(set(target_text.split()[:5]).intersection(set(start_with.split()))) < 2:
            target_text = f"{start_with} {target_text}"

        # Build task input
        task_prefix = (
            f"Adhering to the condition of 'begin summary with' and 'tone of summary' and summarize "
            f"according to {perspective} and start the summary with '{start_with}'. "
            f"Maintain summary tone as {tone}. "
            f"Definition of perspective: {definition}. "
            f"Content to summarize: {concatenated_answers} Question: {question}."
        )

        inputs = self.tokenizer(task_prefix, padding="max_length", max_length=self.max_length,
                                truncation=True, return_tensors="pt")
        labels = self.tokenizer(target_text, padding="max_length", max_length=self.max_length,
                                truncation=True, return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
            "perspective": perspective,
            "Summary": target_text
        }



### =============================== Dataloaders ===============================
def create_dataloader(train_dataset, valid_dataset, train_bs, valid_bs):
    return (
        DataLoader(train_dataset, batch_size=train_bs, shuffle=True),
        DataLoader(valid_dataset, batch_size=valid_bs, shuffle=True)
    )

def test_create_dataloader(test_dataset, test_bs):
    return DataLoader(test_dataset, batch_size=test_bs, shuffle=False)


### =============================== Models ===============================
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5).to(device)

ckpt_path = "./classifier/checkpoint_classifier"
if os.path.exists(ckpt_path):
    print("Loading the trained checkpoint...")
    ckpt = torch.load(ckpt_path)
    roberta_model.load_state_dict(ckpt['model_state_dict'])


### =============================== Embedding & Scoring ===============================
def get_bert_embedding(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()


def Ep(summary):
    inputs = roberta_tokenizer(summary, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = roberta_model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
    classes = ["EXPERIENCE", "SUGGESTION", "INFORMATION", "CAUSE", "QUESTION"]
    return {cls: probs[0][i].item() for i, cls in enumerate(classes)}


def Es(summary):
    start_phrases = [
        "In user's experience…", "It is suggested", "For information purposes",
        "Some of the causes", "It is inquired"
    ]
    pred = ' '.join(summary.split()[:4])
    rouge = Rouge()
    scores = {}
    for ref in start_phrases:
        score = rouge.get_scores(pred.lower(), ref.lower())[0]["rouge-1"]["f"]
        scores[ref] = score
    return scores


def Et(summary):
    tone_dict = {
        'sugg': ["Advisory", "Recommending", "Cautioning", "Prescriptive"],
        'exp': ["Personal", "Narrative", "Introspective"],
        'info': ["Clinical", "Scientific", "Informative"],
        'cause': ["Diagnostic", "Explanatory", "Causal"],
        'qs': ["Inquiry", "Rhetorical", "Exploratory Questioning"]
    }
    summary_emb = get_bert_embedding(summary)
    sims = {}
    for k, word_list in tone_dict.items():
        phrase_emb = get_bert_embedding(' '.join(word_list))
        sims[k] = 1 - cosine(summary_emb.detach().cpu().numpy(), phrase_emb.detach().cpu().numpy())
    return sims



### =============================== Custom Loss ===============================
def compute_custom_loss(model, input_ids, attention_mask, perspectives, tokenizer):
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=100, num_beams=5)
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if len(generated_summary) <= 0:
        generated_summary = 'None'

    Ep_dict = Ep(generated_summary)
    Es_dict = Es(generated_summary)
    Et_dict = Et(generated_summary)

    alpha, beta, gamma = 0.7, 0.3, 0.5

    E_X = {
        "EXPERIENCE": alpha * Ep_dict["EXPERIENCE"] + beta * Es_dict["In user's experience…"] + gamma * Et_dict['exp'],
        "SUGGESTION": alpha * Ep_dict["SUGGESTION"] + beta * Es_dict["It is suggested"] + gamma * Et_dict['sugg'],
        "INFORMATION": alpha * Ep_dict["INFORMATION"] + beta * Es_dict["For information purposes"] + gamma * Et_dict['info'],
        "CAUSE": alpha * Ep_dict["CAUSE"] + beta * Es_dict["Some of the causes"] + gamma * Et_dict['cause'],
        "QUESTION": alpha * Ep_dict["QUESTION"] + beta * Es_dict["It is inquired"] + gamma * Et_dict['qs']
    }

    exp_E_X = {k: math.exp(-1 / (v + 1e-6)) for k, v in E_X.items()}
    Z = sum(exp_E_X.values())
    P_X = {k: v / Z for k, v in exp_E_X.items()}

    Y = {k: 0 for k in E_X}
    Y[perspectives[0]] = 1

    P_X_tensor = torch.tensor(list(P_X.values())).to(device)
    Y_tensor = torch.tensor(list(Y.values())).to(device)

    return -torch.sum(Y_tensor * torch.log(P_X_tensor + 1e-6))


### =============================== Validation Loop ===============================
def validate(model, valid_loader, tokenizer):
    print("Starting validation...")
    model.eval()
    losses = []
    for i, batch in enumerate(tqdm(valid_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            custom_loss = compute_custom_loss(model, input_ids, attention_mask, [batch["perspective"][0]], tokenizer)
            total_loss = output.loss + custom_loss
            losses.append(total_loss.item())

        print(f"Batch {i+1}/{len(valid_loader)} | Loss: {total_loss.item():.4f}")

    avg_loss = np.mean(losses)
    print(f"\nValidation completed. Avg loss: {avg_loss:.4f}")
    return avg_loss


def main():
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    import random

    # Set seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    # -------------------- Load Data --------------------
    with open("/kaggle/input/plasma-data/train.json", "r") as f:
        train_data = json.load(f)
    with open("/kaggle/input/plasma-data/valid.json", "r") as f:
        val_data = json.load(f)

    # -------------------- Model Setup --------------------
    model_name = "facebook/bart-base"  # or use 'google/flan-t5-base'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    # -------------------- Dataset and Dataloader --------------------
    train_dataset = CustomDataset(train_data, tokenizer)
    val_dataset = CustomDataset(val_data, tokenizer)

    train_loader, val_loader = create_dataloader(train_dataset, val_dataset, train_bs=2, valid_bs=2)

    # -------------------- Optimizer --------------------
    optimizer = AdamW(model.parameters(), lr=5e-5)



    num_epochs = 10
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        print(f"\n======== Epoch {epoch + 1}/{num_epochs} ========")
        model.train()
        epoch_losses = []

        for step, batch in enumerate(tqdm(train_loader)):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            perspectives = [batch["perspective"]] if isinstance(batch["perspective"], str) else batch["perspective"]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss_ce = outputs.loss
            # loss_custom = compute_custom_loss(model, input_ids, attention_mask, perspectives, tokenizer)
            loss_custom = compute_custom_loss(model, input_ids[0].unsqueeze(0), attention_mask[0].unsqueeze(0), [batch["perspective"][0]], tokenizer)

            total_loss = loss_ce + loss_custom

            total_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            epoch_losses.append(total_loss.item())

            if step % 10 == 0:
                print(f"Step {step} | Loss: {total_loss.item():.4f} (CE: {loss_ce.item():.4f}, Custom: {loss_custom.item():.4f})")

        avg_train_loss = np.mean(epoch_losses)
        print(f"Epoch {epoch + 1} Avg Training Loss: {avg_train_loss:.4f}")

        # -------------------- Validation --------------------
        val_loss = validate(model, val_loader, tokenizer)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print(f"Saving best model (val_loss = {val_loss:.4f})...")
            model.save_pretrained("best_model")
            tokenizer.save_pretrained("best_model")

    print("\nTraining Finished!")


if __name__ == "__main__":
    main()


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

...

Batch 478/480 | Loss: 1.3200


100%|█████████▉| 479/480 [10:42<00:01,  1.12s/it]

Batch 479/480 | Loss: 1.0021


100%|██████████| 480/480 [10:43<00:00,  1.34s/it]

Batch 480/480 | Loss: 1.5797

Validation completed. Avg loss: 1.2284

Training Finished!


In [8]:
import shutil

# Source path
source_path = "/kaggle/working/best_model"

# Destination path (this will create a zip file you can download)
shutil.make_archive('/kaggle/working/best_model', 'zip', source_path)


'/kaggle/working/best_model.zip'

## 2.1 Bart model : (Inference)

In [9]:
import os
import json
import torch
import pandas as pd
from tqdm import tqdm
from rouge import Rouge
from bert_score import score as bert_scoring
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import Dataset, DataLoader
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item.get("question", "").strip()
        answers = item.get("answers", [])

        labelled_summary_dict = item.get("labelled_summaries", {})
        
        if not labelled_summary_dict:
            return self.__getitem__((idx + 1) % len(self.data))  # Skip bad sample


        labelled_answer_spans = item.get("labelled_answer_spans", {})

        
        # Assume only one perspective (e.g., "INFORMATION")
        # if not labelled_summary_dict:
        #     raise ValueError("Missing labelled_summaries in example.")
        perspective_key = list(labelled_summary_dict.keys())[0]
        perspective = perspective_key.replace("_SUMMARY", "")
        target_text = labelled_summary_dict[perspective_key].strip()

        # Prepare answer context
        concatenated_answers = " ".join([ans.replace('\n', ' ').strip() for ans in answers])

        # Definitions and tones for guidance
        start_phrases = {
            "SUGGESTION": ("It is suggested", "Advisory, Recommending",
                           ["Advisory", "Recommending", "Cautioning", "Prescriptive", "Guiding"]),
            "INFORMATION": ("For information purposes", "Informative, Educational",
                            ["Clinical", "Scientific", "Informative", "Educational"]),
            "EXPERIENCE": ("In user's experience", "Personal, Narrative",
                           ["Personal", "Narrative", "Introspective", "Exemplary"]),
            "CAUSE": ("Some of the causes", "Explanatory, Causal",
                      ["Diagnostic", "Explanatory", "Causal", "Due to"]),
            "QUESTION": ("It is inquired", "Seeking Understanding",
                         ["Inquiry", "Rhetorical", "Exploratory Questioning"])
        }

        definitions = {
            "SUGGESTION": "Advice or recommendations to assist users.",
            "INFORMATION": "Knowledge about diseases and facts.",
            "EXPERIENCE": "Individual experiences or insights.",
            "CAUSE": "Reasons responsible for symptoms or conditions.",
            "QUESTION" : "Inquiry made for deeper understanding."
        }

        start_with, tone, _ = start_phrases.get(perspective, ("", "", []))
        definition = definitions.get(perspective, "")

        # Check and prepend start phrase if necessary
        if len(set(target_text.split()[:5]).intersection(set(start_with.split()))) < 2:
            target_text = f"{start_with} {target_text}"

        # Build task input
        task_prefix = (
            f"Adhering to the condition of 'begin summary with' and 'tone of summary' and summarize "
            f"according to {perspective} and start the summary with '{start_with}'. "
            f"Maintain summary tone as {tone}. "
            f"Definition of perspective: {definition}. "
            f"Content to summarize: {concatenated_answers} Question: {question}."
        )

        inputs = self.tokenizer(task_prefix, padding="max_length", max_length=self.max_length,
                                truncation=True, return_tensors="pt")
        labels = self.tokenizer(target_text, padding="max_length", max_length=self.max_length,
                                truncation=True, return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
            "perspective": perspective,
            "Summary": target_text
        }



### =============================== Dataloaders ===============================
def create_dataloader(train_dataset, valid_dataset, train_bs, valid_bs):
    return (
        DataLoader(train_dataset, batch_size=train_bs, shuffle=True),
        DataLoader(valid_dataset, batch_size=valid_bs, shuffle=True)
    )

def test_create_dataloader(test_dataset, test_bs):
    return DataLoader(test_dataset, batch_size=test_bs, shuffle=False)


def calculate_metrics(pred_summary, actual_summary):
    # Tokenize for METEOR and BLEU
    pred_tokens = pred_summary.split()
    actual_tokens = [actual_summary.split()]  # Note: BLEU expects reference as list of lists
    
    # Calculate BLEU
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(actual_tokens, pred_tokens, smoothing_function=smoothie)
    
    # Calculate ROUGE
    rouge = Rouge()
    rouge_scores = rouge.get_scores(pred_summary, actual_summary)[0]
    
    # Calculate METEOR
    meteor = meteor_score(actual_tokens, pred_tokens)
    
    return {
        'BLEU': bleu_score,
        'ROUGE-1': rouge_scores['rouge-1']['f'],
        'ROUGE-2': rouge_scores['rouge-2']['f'],
        'ROUGE-L': rouge_scores['rouge-l']['f'],
        'METEOR': meteor
    }

def run_inference_with_metrics(model, test_loader, tokenizer):
    model.eval()
    results = []
    all_preds = []
    all_refs = []
    
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        perspectives = batch['perspective']
        actual_summaries = batch['Summary']
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=150,
                num_beams=5,
                early_stopping=True
            )
        
        for i in range(len(outputs)):
            pred_summary = tokenizer.decode(outputs[i], skip_special_tokens=True)
            actual_summary = actual_summaries[i]
            perspective = perspectives[i]
            
            metrics = calculate_metrics(pred_summary, actual_summary)
            
            result = {
                "Perspective": perspective,
                "Actual Summary": actual_summary,
                "Predicted Summary": pred_summary,
                **metrics
            }
            results.append(result)
            all_preds.append(pred_summary)
            all_refs.append(actual_summary)
    
    # Calculate BERTScore for all samples at once (more efficient)
    P, R, F1 = bert_scoring(all_preds, all_refs, lang='en', verbose=True)
    
    # Add BERTScore to individual results
    for i, result in enumerate(results):
        result['BERTScore_P'] = P[i].item()
        result['BERTScore_R'] = R[i].item()
        result['BERTScore_F1'] = F1[i].item()
    
    return results, (P.mean().item(), R.mean().item(), F1.mean().item())

def calculate_perspective_wise_metrics(results):
    perspectives = set(r['Perspective'] for r in results)
    perspective_metrics = {}
    
    for perspective in perspectives:
        perspective_results = [r for r in results if r['Perspective'] == perspective]
        
        metrics = {
            'Count': len(perspective_results),
            'R1': sum(r['ROUGE-1'] for r in perspective_results) / len(perspective_results) * 100,
            'R2': sum(r['ROUGE-2'] for r in perspective_results) / len(perspective_results) * 100,
            'RL': sum(r['ROUGE-L'] for r in perspective_results) / len(perspective_results) * 100,
            'BERTScore': sum(r['BERTScore_F1'] for r in perspective_results) / len(perspective_results),
            'METEOR': sum(r['METEOR'] for r in perspective_results) / len(perspective_results),
            'BLEU': sum(r['BLEU'] for r in perspective_results) / len(perspective_results)
        }
        perspective_metrics[perspective] = metrics
    
    return perspective_metrics

def save_perspective_wise_table(perspective_metrics, filename="perspective_wise_metrics.csv"):
    # Prepare data for DataFrame
    data = []
    for perspective, metrics in perspective_metrics.items():
        data.append({
            'Perspective': perspective,
            'R1': f"{metrics['R1']:.2f}",
            'R2': f"{metrics['R2']:.2f}",
            'RL': f"{metrics['RL']:.2f}",
            'BERTScore': f"{metrics['BERTScore']:.3f}",
            'METEOR': f"{metrics['METEOR']:.3f}",
            'BLEU': f"{metrics['BLEU']:.3f}"
        })
    
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Perspective-wise metrics saved to {filename}")
    return df



def print_perspective_wise_table(perspective_metrics):
    print("\nPERSPECTIVE-WISE METRICS:")
    print("{:<15} {:<8} {:<8} {:<8} {:<10} {:<8} {:<8}".format(
        "Perspective", "R1", "R2", "RL", "BERTScore", "METEOR", "BLEU"))
    print("-"*65)
    
    for perspective, metrics in perspective_metrics.items():
        print("{:<15} {:<8.2f} {:<8.2f} {:<8.2f} {:<10.3f} {:<8.3f} {:<8.3f}".format(
            perspective,
            metrics['R1'],
            metrics['R2'],
            metrics['RL'],
            metrics['BERTScore'],
            metrics['METEOR'],
            metrics['BLEU']
        ))

def save_all_results_json(results, filename="all_evaluation_results.json"):
    """Save all evaluation results in JSON format"""
    # Convert DataFrame-compatible results to JSON-serializable format
    json_results = []
    for result in results:
        json_result = {
            "Perspective": result["Perspective"],
            "Actual_Summary": result["Actual Summary"],
            "Predicted_Summary": result["Predicted Summary"],
            "Metrics": {
                "BLEU": float(result["BLEU"]),
                "ROUGE-1": float(result["ROUGE-1"]),
                "ROUGE-2": float(result["ROUGE-2"]),
                "ROUGE-L": float(result["ROUGE-L"]),
                "METEOR": float(result["METEOR"]),
                "BERTScore_P": float(result["BERTScore_P"]),
                "BERTScore_R": float(result["BERTScore_R"]),
                "BERTScore_F1": float(result["BERTScore_F1"])
            }
        }
        json_results.append(json_result)
    
    with open(filename, 'w') as f:
        json.dump(json_results, f, indent=4)
    print(f"All evaluation results saved to {filename}")

def save_perspective_metrics_json(perspective_metrics, filename="perspective_wise_metrics.json"):
    """Save perspective-wise metrics in JSON format"""
    # Convert metrics to JSON-serializable format
    json_metrics = {}
    for perspective, metrics in perspective_metrics.items():
        json_metrics[perspective] = {
            "R1": float(metrics['R1']),
            "R2": float(metrics['R2']),
            "RL": float(metrics['RL']),
            "BERTScore": float(metrics['BERTScore']),
            "METEOR": float(metrics['METEOR']),
            "BLEU": float(metrics['BLEU']),
            "Count": int(metrics['Count'])
        }
    
    with open(filename, 'w') as f:
        json.dump(json_metrics, f, indent=4)
    print(f"Perspective-wise metrics saved to {filename}")


# Load test data
with open("/kaggle/input/plasma-dat/test.json", "r") as f:
    test_data = json.load(f)

# Initialize model and tokenizer
model_name = "/kaggle/input/best_model/transformers/default/1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Create test dataset and loader
test_dataset = CustomDataset(test_data, tokenizer)
test_loader = test_create_dataloader(test_dataset, test_bs=4)

results, overall_bertscore = run_inference_with_metrics(model, test_loader, tokenizer)

# Calculate perspective-wise metrics
perspective_metrics = calculate_perspective_wise_metrics(results)

# Save results in JSON format
save_all_results_json(results)  # Saves all individual evaluation results
save_perspective_metrics_json(perspective_metrics)  # Saves perspective-wise metrics

# Also save in CSV format (optional)
all_results_df = pd.DataFrame(results)
all_results_df.to_csv("all_evaluation_results.csv", index=False)

perspective_df = pd.DataFrame.from_dict(perspective_metrics, orient='index')
perspective_df.to_csv("perspective_wise_metrics.csv")

# Print results
print_perspective_wise_table(perspective_metrics)

100%|██████████| 160/160 [05:13<00:00,  1.96s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/20 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/10 [00:00<?, ?it/s]

done in 15.34 seconds, 41.73 sentences/sec
All evaluation results saved to all_evaluation_results.json
Perspective-wise metrics saved to perspective_wise_metrics.json

PERSPECTIVE-WISE METRICS:
Perspective     R1       R2       RL       BERTScore  METEOR   BLEU    
-----------------------------------------------------------------
EXPERIENCE      38.96    25.59    37.27    0.902      0.351    0.162   
QUESTION        53.37    36.87    52.93    0.919      0.475    0.231   
INFORMATION     38.64    18.57    36.17    0.894      0.301    0.094   
SUGGESTION      33.74    17.87    31.73    0.888      0.248    0.088   
CAUSE           41.56    25.35    39.43    0.897      0.359    0.158   


# 2.2 Bart Large Model :

In [ ]:
import os
import json
import math
import torch
import warnings
import numpy as np
from tqdm import tqdm
from rouge import Rouge
from scipy.spatial.distance import cosine
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import (
    BertTokenizer, BertModel, RobertaTokenizer, RobertaForSequenceClassification,
    AutoTokenizer, AutoModelForSeq2SeqLM
)
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW


warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### =============================== Dataset ===============================
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item.get("question", "").strip()
        answers = item.get("answers", [])

        labelled_summary_dict = item.get("labelled_summaries", {})
        
        if not labelled_summary_dict:
            return self.__getitem__((idx + 1) % len(self.data))  # Skip bad sample


        labelled_answer_spans = item.get("labelled_answer_spans", {})

        
        # Assume only one perspective (e.g., "INFORMATION")
        # if not labelled_summary_dict:
        #     raise ValueError("Missing labelled_summaries in example.")
        perspective_key = list(labelled_summary_dict.keys())[0]
        perspective = perspective_key.replace("_SUMMARY", "")
        target_text = labelled_summary_dict[perspective_key].strip()

        # Prepare answer context
        concatenated_answers = " ".join([ans.replace('\n', ' ').strip() for ans in answers])

        # Definitions and tones for guidance
        start_phrases = {
            "SUGGESTION": ("It is suggested", "Advisory, Recommending",
                           ["Advisory", "Recommending", "Cautioning", "Prescriptive", "Guiding"]),
            "INFORMATION": ("For information purposes", "Informative, Educational",
                            ["Clinical", "Scientific", "Informative", "Educational"]),
            "EXPERIENCE": ("In user's experience", "Personal, Narrative",
                           ["Personal", "Narrative", "Introspective", "Exemplary"]),
            "CAUSE": ("Some of the causes", "Explanatory, Causal",
                      ["Diagnostic", "Explanatory", "Causal", "Due to"]),
            "QUESTION": ("It is inquired", "Seeking Understanding",
                         ["Inquiry", "Rhetorical", "Exploratory Questioning"])
        }

        definitions = {
            "SUGGESTION": "Advice or recommendations to assist users.",
            "INFORMATION": "Knowledge about diseases and facts.",
            "EXPERIENCE": "Individual experiences or insights.",
            "CAUSE": "Reasons responsible for symptoms or conditions.",
            "QUESTION" : "Inquiry made for deeper understanding."
        }

        start_with, tone, _ = start_phrases.get(perspective, ("", "", []))
        definition = definitions.get(perspective, "")

        # Check and prepend start phrase if necessary
        if len(set(target_text.split()[:5]).intersection(set(start_with.split()))) < 2:
            target_text = f"{start_with} {target_text}"

        # Build task input
        task_prefix = (
            f"Adhering to the condition of 'begin summary with' and 'tone of summary' and summarize "
            f"according to {perspective} and start the summary with '{start_with}'. "
            f"Maintain summary tone as {tone}. "
            f"Definition of perspective: {definition}. "
            f"Content to summarize: {concatenated_answers} Question: {question}."
        )

        inputs = self.tokenizer(task_prefix, padding="max_length", max_length=self.max_length,
                                truncation=True, return_tensors="pt")
        labels = self.tokenizer(target_text, padding="max_length", max_length=self.max_length,
                                truncation=True, return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
            "perspective": perspective,
            "Summary": target_text
        }



### =============================== Dataloaders ===============================
def create_dataloader(train_dataset, valid_dataset, train_bs, valid_bs):
    return (
        DataLoader(train_dataset, batch_size=train_bs, shuffle=True),
        DataLoader(valid_dataset, batch_size=valid_bs, shuffle=True)
    )

def test_create_dataloader(test_dataset, test_bs):
    return DataLoader(test_dataset, batch_size=test_bs, shuffle=False)


### =============================== Models ===============================
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5).to(device)

ckpt_path = "./classifier/checkpoint_classifier"
if os.path.exists(ckpt_path):
    print("Loading the trained checkpoint...")
    ckpt = torch.load(ckpt_path)
    roberta_model.load_state_dict(ckpt['model_state_dict'])


### =============================== Embedding & Scoring ===============================
def get_bert_embedding(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()


def Ep(summary):
    inputs = roberta_tokenizer(summary, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = roberta_model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
    classes = ["EXPERIENCE", "SUGGESTION", "INFORMATION", "CAUSE", "QUESTION"]
    return {cls: probs[0][i].item() for i, cls in enumerate(classes)}


def Es(summary):
    if not summary.strip():  # If summary is empty after stripping whitespace
        return {ref: 0.0 for ref in [
            "In user's experience…", "It is suggested", "For information purposes",
            "Some of the causes", "It is inquired"
        ]}
    
    start_phrases = [
        "In user's experience…", "It is suggested", "For information purposes",
        "Some of the causes", "It is inquired"
    ]
    pred = ' '.join(summary.split()[:4])
    rouge = Rouge()
    scores = {}
    for ref in start_phrases:
        try:
            score = rouge.get_scores(pred.lower(), ref.lower())[0]["rouge-1"]["f"]
        except ValueError:  # In case of any ROUGE calculation error
            score = 0.0
        scores[ref] = score
    return scores


def Et(summary):
    tone_dict = {
        'sugg': ["Advisory", "Recommending", "Cautioning", "Prescriptive"],
        'exp': ["Personal", "Narrative", "Introspective"],
        'info': ["Clinical", "Scientific", "Informative"],
        'cause': ["Diagnostic", "Explanatory", "Causal"],
        'qs': ["Inquiry", "Rhetorical", "Exploratory Questioning"]
    }
    summary_emb = get_bert_embedding(summary)
    sims = {}
    for k, word_list in tone_dict.items():
        phrase_emb = get_bert_embedding(' '.join(word_list))
        sims[k] = 1 - cosine(summary_emb.detach().cpu().numpy(), phrase_emb.detach().cpu().numpy())
    return sims



### =============================== Custom Loss ===============================
def compute_custom_loss(model, input_ids, attention_mask, perspectives, tokenizer):
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=100, num_beams=5)
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if not generated_summary.strip():  # If empty summary
        return torch.tensor(0.0, device=device)  # Return zero loss for empty summaries

    try:
        Ep_dict = Ep(generated_summary)
        Es_dict = Es(generated_summary)
        Et_dict = Et(generated_summary)
    except Exception as e:
        print(f"Error calculating custom metrics: {e}")
        return torch.tensor(0.0, device=device)

    alpha, beta, gamma = 0.7, 0.3, 0.5

    E_X = {
        "EXPERIENCE": alpha * Ep_dict["EXPERIENCE"] + beta * Es_dict["In user's experience…"] + gamma * Et_dict['exp'],
        "SUGGESTION": alpha * Ep_dict["SUGGESTION"] + beta * Es_dict["It is suggested"] + gamma * Et_dict['sugg'],
        "INFORMATION": alpha * Ep_dict["INFORMATION"] + beta * Es_dict["For information purposes"] + gamma * Et_dict['info'],
        "CAUSE": alpha * Ep_dict["CAUSE"] + beta * Es_dict["Some of the causes"] + gamma * Et_dict['cause'],
        "QUESTION": alpha * Ep_dict["QUESTION"] + beta * Es_dict["It is inquired"] + gamma * Et_dict['qs']
    }

    exp_E_X = {k: math.exp(-1 / (v + 1e-6)) for k, v in E_X.items()}
    Z = sum(exp_E_X.values())
    P_X = {k: v / Z for k, v in exp_E_X.items()}

    Y = {k: 0 for k in E_X}
    Y[perspectives[0]] = 1

    P_X_tensor = torch.tensor(list(P_X.values())).to(device)
    Y_tensor = torch.tensor(list(Y.values())).to(device)

    return -torch.sum(Y_tensor * torch.log(P_X_tensor + 1e-6))


### =============================== Validation Loop ===============================
def validate(model, valid_loader, tokenizer):
    print("Starting validation...")
    model.eval()
    losses = []
    for i, batch in enumerate(tqdm(valid_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            custom_loss = compute_custom_loss(model, input_ids, attention_mask, [batch["perspective"][0]], tokenizer)
            total_loss = output.loss + custom_loss
            losses.append(total_loss.item())

        print(f"Batch {i+1}/{len(valid_loader)} | Loss: {total_loss.item():.4f}")

    avg_loss = np.mean(losses)
    print(f"\nValidation completed. Avg loss: {avg_loss:.4f}")
    return avg_loss


def main():
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    import random

    # Set seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    # -------------------- Load Data --------------------
    with open("/kaggle/input/plasma-dat/train.json", "r") as f:
        train_data = json.load(f)
    with open("/kaggle/input/plasma-dat/valid.json", "r") as f:
        val_data = json.load(f)

    # -------------------- Model Setup --------------------
    model_name = "facebook/bart-large"  # or use 'google/flan-t5-base'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    # -------------------- Dataset and Dataloader --------------------
    train_dataset = CustomDataset(train_data, tokenizer)
    val_dataset = CustomDataset(val_data, tokenizer)

    train_loader, val_loader = create_dataloader(train_dataset, val_dataset, train_bs=2, valid_bs=2)

    # -------------------- Optimizer --------------------
    optimizer = AdamW(model.parameters(), lr=5e-5)



    num_epochs = 5
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        print(f"\n======== Epoch {epoch + 1}/{num_epochs} ========")
        model.train()
        epoch_losses = []

        for step, batch in enumerate(tqdm(train_loader)):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            perspectives = [batch["perspective"]] if isinstance(batch["perspective"], str) else batch["perspective"]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss_ce = outputs.loss
            # loss_custom = compute_custom_loss(model, input_ids, attention_mask, perspectives, tokenizer)
            loss_custom = compute_custom_loss(model, input_ids[0].unsqueeze(0), attention_mask[0].unsqueeze(0), [batch["perspective"][0]], tokenizer)

            total_loss = loss_ce + loss_custom

            total_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            epoch_losses.append(total_loss.item())

            if step % 10 == 0:
                print(f"Step {step} | Loss: {total_loss.item():.4f} (CE: {loss_ce.item():.4f}, Custom: {loss_custom.item():.4f})")

        avg_train_loss = np.mean(epoch_losses)
        print(f"Epoch {epoch + 1} Avg Training Loss: {avg_train_loss:.4f}")

        # -------------------- Validation --------------------
        val_loss = validate(model, val_loader, tokenizer)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print(f"Saving best model (val_loss = {val_loss:.4f})...")
            model.save_pretrained("best_model")
            tokenizer.save_pretrained("best_model")

    print("\nTraining Finished!")


if __name__ == "__main__":
    main()


2025-04-14 05:27:28.568260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744608448.963859      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744608449.069650      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

 31%|███▏      | 351/1118 [22:12<50:37,  3.96s/it]

Step 350 | Loss: 2.3329 (CE: 1.0418, Custom: 1.2911)


 32%|███▏      | 361/1118 [22:45<40:26,  3.21s/it]

Step 360 | Loss: 1.8808 (CE: 0.5481, Custom: 1.3327)


 33%|███▎      | 369/1118 [23:11<39:52,  3.19s/it]

## 2.2 Bart Large Model : (Inference)

In [4]:
import json
import numpy as np
import zipfile
import os
from tqdm import tqdm
from rouge import Rouge
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from bert_score import score as bert_score
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')

# Initialize metrics
rouge = Rouge()
smoother = SmoothingFunction()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model_path = "/kaggle/input/best_model/transformers/default/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
model.eval()

# Load test data
with open("/kaggle/input/plasma-dat/test.json", "r") as f:
    test_data = json.load(f)

# Prepare for evaluation
all_results = []
perspective_metrics = {
    "EXPERIENCE": {"R1": [], "R2": [], "RL": [], "BERTScore": [], "METEOR": [], "BLEU": []},
    "QUESTION": {"R1": [], "R2": [], "RL": [], "BERTScore": [], "METEOR": [], "BLEU": []},
    "INFORMATION": {"R1": [], "R2": [], "RL": [], "BERTScore": [], "METEOR": [], "BLEU": []},
    "SUGGESTION": {"R1": [], "R2": [], "RL": [], "BERTScore": [], "METEOR": [], "BLEU": []},
    "CAUSE": {"R1": [], "R2": [], "RL": [], "BERTScore": [], "METEOR": [], "BLEU": []}
}

def compute_metrics(pred, ref, perspective):
    # ROUGE scores
    try:
        rouge_scores = rouge.get_scores(pred, ref)[0]
        r1 = rouge_scores['rouge-1']['f'] * 100
        r2 = rouge_scores['rouge-2']['f'] * 100
        rl = rouge_scores['rouge-l']['f'] * 100
    except:
        r1, r2, rl = 0, 0, 0

    # BERTScore
    P, R, F1 = bert_score([pred], [ref], lang='en', verbose=False)
    bertscore = F1.mean().item() * 100

    # METEOR
    meteor = meteor_score([word_tokenize(ref)], word_tokenize(pred)) * 100

    # BLEU
    bleu = sentence_bleu([word_tokenize(ref)], word_tokenize(pred),
                         smoothing_function=smoother.method1) * 100

    # Store metrics
    perspective_metrics[perspective]["R1"].append(r1)
    perspective_metrics[perspective]["R2"].append(r2)
    perspective_metrics[perspective]["RL"].append(rl)
    perspective_metrics[perspective]["BERTScore"].append(bertscore)
    perspective_metrics[perspective]["METEOR"].append(meteor)
    perspective_metrics[perspective]["BLEU"].append(bleu)

    return {
        "ROUGE-1": r1,
        "ROUGE-2": r2,
        "ROUGE-L": rl,
        "BERTScore": bertscore,
        "METEOR": meteor,
        "BLEU": bleu
    }

# Run inference and evaluation
for item in tqdm(test_data, desc="Evaluating"):
    question = item.get("question", "").strip()
    answers = item.get("answers", [])
    concatenated_answers = " ".join([ans.replace('\n', ' ').strip() for ans in answers])

    labelled_summary_dict = item.get("labelled_summaries", {})
    if not labelled_summary_dict:
        continue

    perspective_key = list(labelled_summary_dict.keys())[0]
    perspective = perspective_key.replace("_SUMMARY", "")
    target_summary = labelled_summary_dict[perspective_key].strip()

    # Prepare input
    task_prefix = (
        f"Adhering to the condition of 'begin summary with' and 'tone of summary' and summarize "
        f"according to {perspective}. Content to summarize: {concatenated_answers} Question: {question}."
    )

    inputs = tokenizer(task_prefix, return_tensors="pt", truncation=True, max_length=1024).to(device)

    # Generate summary
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, num_beams=5)
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Compute metrics
    metrics = compute_metrics(generated_summary, target_summary, perspective)

    # Store results
    all_results.append({
        "question": question,
        "answers": answers,
        "perspective": perspective,
        "generated_summary": generated_summary,
        "target_summary": target_summary,
        "metrics": metrics
    })

# Calculate average metrics per perspective
final_perspective_metrics = {}
for perspective, metrics in perspective_metrics.items():
    if metrics["R1"]:  # Only if we have samples for this perspective
        final_perspective_metrics[perspective] = {
            "R1": np.mean(metrics["R1"]),
            "R2": np.mean(metrics["R2"]),
            "RL": np.mean(metrics["RL"]),
            "BERTScore": np.mean(metrics["BERTScore"]),
            "METEOR": np.mean(metrics["METEOR"]),
            "BLEU": np.mean(metrics["BLEU"])
        }

# Save results
with open("all_evaluation_results.json", "w") as f:
    json.dump(all_results, f, indent=2)

with open("perspective_wise_metrics.json", "w") as f:
    json.dump(final_perspective_metrics, f, indent=2)

# Print the formatted table
print("\nPERSPECTIVE-WISE METRICS:")
print("{:<12} {:<8} {:<8} {:<8} {:<10} {:<8} {:<8}".format(
    "Perspective", "R1", "R2", "RL", "BERTScore", "METEOR", "BLEU"))
print("-" * 65)
for perspective, metrics in final_perspective_metrics.items():
    print("{:<12} {:<8.2f} {:<8.2f} {:<8.2f} {:<10.3f} {:<8.3f} {:<8.3f}".format(
        perspective,
        metrics["R1"],
        metrics["R2"],
        metrics["RL"],
        metrics["BERTScore"]/100,  # Convert back to 0-1 scale for display
        metrics["METEOR"]/100,
        metrics["BLEU"]/100
    ))

print("\nAll evaluation results saved to all_evaluation_results.json")
print("Perspective-wise metrics saved to perspective_wise_metrics.json")

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2025-04-14 19:42:54.612291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744659774.959371      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744659775.047744      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Evaluating:   0%|          | 0/640 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:   0%|          | 1/640 [00:34<6:09:09, 34.66s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:   0%|          | 2/640 [00:45<3:38:59, 20.59s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:   0%|          | 3/640 [00:51<2:27:00,


PERSPECTIVE-WISE METRICS:
Perspective  R1       R2       RL       BERTScore  METEOR   BLEU    
-----------------------------------------------------------------
EXPERIENCE   14.68    3.42     12.74    0.849      0.141    0.014   
QUESTION     10.86    1.91     9.59     0.844      0.141    0.012   
INFORMATION  30.19    12.92    27.33    0.881      0.252    0.071   
SUGGESTION   21.29    6.08     18.75    0.870      0.183    0.030   
CAUSE        24.38    11.93    22.81    0.879      0.248    0.092   

All evaluation results saved to all_evaluation_results.json
Perspective-wise metrics saved to perspective_wise_metrics.json
